## Loading up packages

In [186]:
# import plastid
# data structure for mapping read alignments to genomic positions
from plastid import BAMGenomeArray, VariableFivePrimeMapFactory, \
                        GTF2_TranscriptAssembler, Transcript, ThreePrimeMapFactory
import numpy as np
import numpy
import pandas as pd
import warnings
import csv
from scipy.sparse.linalg import lsqr

In [253]:
save_path = "/Users/keeganflanagan/Desktop/Khanh_position/Eggtart/"

# The Plan

Wait wait wait, why is this all in codons? I had this set up on a per nucleotide position basis but now it is asking for this on a per codon basis. I guess it does not really matter for this first test but this is something I need to ask Khanh about. 

Ok, so first I am going to take each position in my table and divide that position by the total number of transcripts... when he says transcripts does he really mean reads? It makes no sense to just use whatever number of transcripts happen to be in the genome, it would be much more sensical to use the sum of all of the reads for a transcript. 

Perhaps step 2 is just referring to calculating the average p per 10 codons? So, add up all 10 p values, then divide by 1/10. when he says it should add up to n-9, does that imply that we are finding the average for codons 1-10, then 2-11, then 3-12 all the way up until we can no longer take the average of 10 codons? 

Then, for each set of 10 codons (x entry) calculate the smoothed elongation rate using a pretty straight forward equation. 

Then calculate the scaled initiation rate and termination rate using fairly simple equations and the pbar values calculated for the first and last set of codons. 

Then, you create an array of size n-8-L (n is the length of the gene in codons and L is related to bias and will be set as 1 for now) which contains 10* lambda(x) (the smoother elongation rate for the entries) for which x is between L and n-9. 

Can someone please explain to me what n is in this context? like n is usually the number of something but I cannot figure out which number in this context. Is it always equal to the gene length in codons? 

# Loading up the csv file.

In [172]:
data = []
with open("toy_data.csv", newline = '') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        data.append(row)

In [173]:
blank=data.pop(0)

In [174]:
for i,ii in zip(data, range(len(data))):
    for j,jj in zip(i, range(len(i))):
        try:
            x = int(j)
            data[ii][jj] = x
        except:
            pass

In [175]:
for i,ii in zip(data, range(len(data))):
    x = list(filter(('').__ne__, i))
    data[ii] = x

In [176]:
for i,ii in zip(data, range(len(data))):
    data[ii] = np.array(data[ii][2:])

## Perform the calculations

In [178]:
# Create a function to obtain a normalized profile (p) of ribosome footprints.
def calculate_p(data):
    p_list=[]
    for i in data:
        M = sum(i)
        p = i/M
        p_list.append(p)
    return(p_list)

# Note that I am assuming here that by number of transcripts they meant number of reads.

In [179]:
p_list = calculate_p(data)

In [180]:
# Calculate the smoothed density vector pbar for xth entry with length n-9
def calculate_pbar(p_list):
    pbar_list=[]
    for p in p_list:
        x=0
        pbar=[]
        for px in p:
            pbar_x = 0.1*sum(p[x:x+10]) #it is x+10 not x+9 because python does not include the final index.
            pbar.append(pbar_x)
            x = x+1
            if x  == len(p)-9:
                break
        pbar_list.append(np.array(pbar))
    return(pbar_list)



In [181]:
pbar_list=calculate_pbar(p_list)

In [182]:
# calculate the smoothed, scaled elongation rate lambda bar 
def calculate_lbar(pbar_list):
    lbar_list=[]
    for pbar in pbar_list:
        lbar = []
        for pbarx in pbar:
            lbar_x = (1-9*pbarx)/(pbarx*(1-pbarx))
            lbar.append(lbar_x)
        lbar_list.append(np.array(lbar))
    return(lbar_list)
            

In [183]:
lbar_list=calculate_lbar(pbar_list)

In [169]:
# calculate the scaled initiation and termination rates

init_r= []
for pbar in pbar_list:
    init_r.append(1/(1-10*pbar[0]))

term_r = []
for p in p_list:
    term_r.append(1/p[-1])


I think my initiation and termination rates look very strange because I tended to put very few reads at the beginning of my toy data so it thinks it is going super fast through that section. 

In [237]:
# L = 1 in this case so b is simply lbar list times 10. 
b = 10*lbar_list[0]
A = np.zeros((len(lbar_list[0]),len(p_list[0])))# I cannot seem to understand what A looks like yet.
x=0
for row in A:
    row[x:x+10].fill(1)
    x = x+1

test=lsqr(A,b)

# Perhaps A is meant to be a matrix of size n-8-L*n-8-L,
#### Or is it a mtrix of size n-8-L*n????? Or do I invert that matrix? 
#### like each row corresponds to an x? I think that is right based on row labeled by 𝑥
# with each row corresponding to a different row of X 
# Then, there would be a bunch 10 point long lists of ones. 

In [239]:
Ci = test[0]

In [244]:
tau = Ci.mean()
prod_r = 1/tau

In [247]:
# Computing the unscaled rates
elongation = lbar_list[0]/tau
termination = term_r[0]/tau
initiation = init_r[0]/tau

In [251]:
elongation

array([2.87356524, 2.68598458, 2.28400237, 2.32785689, 1.11079755,
       0.78999285, 0.77763281, 0.77157608, 0.62108648, 0.55609064,
       0.45467921, 0.40995789, 0.36409304, 0.34816587, 0.41656488,
       0.46768923, 0.37339945, 0.30877819, 0.30733429, 0.33321479,
       0.34645814, 0.34988564, 0.33321479, 0.32374584, 0.36046441,
       0.37151073, 0.36777476, 0.42105876, 0.38702221, 0.34307829,
       0.32529762])

In [252]:
# Saving the elongation rate as a csv file. 


array([2.87356524, 2.68598458, 2.28400237, 2.32785689, 1.11079755,
       0.78999285, 0.77763281, 0.77157608, 0.62108648, 0.55609064,
       0.45467921, 0.40995789, 0.36409304, 0.34816587, 0.41656488,
       0.46768923, 0.37339945, 0.30877819, 0.30733429, 0.33321479,
       0.34645814, 0.34988564, 0.33321479, 0.32374584, 0.36046441,
       0.37151073, 0.36777476, 0.42105876, 0.38702221, 0.34307829,
       0.32529762])

In [260]:
with open(save_path + 'keeg_gene_test.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(np.atleast_2d(elongation).T)

How it egg tart calculating the initiation and termination rates? at the moment it produces numbers totally different from what I calculated. Am I supposed to enter in what I calculated? Am I supposed to attach these to the start and end of the elongation file?